# Libraries & Settings

In [2]:
%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np
import qgrid
#import beakerx as bx
from datetime import datetime as dtt

from pa_lib.file import data_files, load_bin, store_bin, store_excel
from pa_lib.data import (calc_col_partitioned, clean_up_categoricals, flatten, 
                         replace_col, cond_col, desc_col, unfactorize)
from pa_lib.util import obj_size, cap_words
from pa_lib.log  import time_log, info
from pa_lib.types import dtFactor
from pa_lib.vis import dive

# display long columns completely
pd.set_option('display.max_colwidth', 200)

pd.set_option('display.max_rows', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load data

In [3]:
bd_raw = load_bin('bd_data.feather').rename(mapper=lambda name: cap_words(name, sep='_'), axis='columns')
bd = bd_raw.loc[(bd_raw.Netto >= 0)].pipe(clean_up_categoricals)

2019-08-06 09:53:36 [INFO] Reading from file /home/pa/data/bd_data.feather
/home/pa/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
2019-08-06 09:53:37 [INFO] Finished loading binary file in 0.99s (0.66s CPU)


In [4]:
desc_col(bd)

,DTYPE,NULLS,UNIQUE
Endkunde_NR,category,0/1537638,41944
Endkunde,category,0/1537638,39314
EK_Abc,category,5386/1532252,8
EK_Boni,category,2408/1535230,3
EK_Plz,category,477/1537161,3474
EK_Ort,category,476/1537162,3509
EK_Land,category,476/1537162,60
EK_HB_Apg_Kurzz,category,67769/1469869,99
EK_Aktiv,category,0/1537638,2
Agentur,category,775808/761830,4638


# Prepare Endkunden Information

In [7]:
def collect(s, sep=', '):
    return sep.join(map(str, s[s.notna()].unique()))

ek_info = (bd.sort_values(['Endkunde_NR', 'Kampagne_Erfassungsdatum'])
           .groupby('Endkunde_NR')
           .agg({'Endkunde': 'last',
                 'EK_Aktiv': 'last',
                 'EK_Land': 'last',
                 'EK_Plz': 'last',
                 'EK_Ort': 'last',
                 'Agentur': 'last',
                 'Endkunde_Branchengruppe': 'last',
                 'Endkunde_Branchengruppe_ID': 'last',
                 'Auftrag_Branchengruppe_ID': [collect, 'nunique'],
                 'Kamp_Erfass_Jahr': ['min', 'max']
                 }))

ek_info

Endkunde EK_Aktiv  EK_Land EK_Plz  \
                                     last     last     last   last   
0                    Maurer + Salzmann AG        1  SCHWEIZ   8408   
1                 Bank BSU Genossenschaft        1  SCHWEIZ   8610   
2       Werbeagentur Knaus & Knaus BSW AG        0  SCHWEIZ   8604   
3                    Stoz Werbeagentur AG        1  SCHWEIZ   8330   
4                    Mundwiler Juwelen AG        1  SCHWEIZ   8400   
...                                   ...      ...      ...    ...   
657692                                NaN      NaN      NaN    NaN   
657698                                NaN      NaN      NaN    NaN   
657704                                NaN      NaN      NaN    NaN   
657718                                NaN      NaN      NaN    NaN   
657720                                NaN      NaN      NaN    NaN   

              EK_Ort                         Agentur  \
                last                            last   
0         Winterthur            Maurer + Salzmann AG   
1              Uster  walder, werber werbeagentur ag   
2         Volketswil                             NaN   
3       Pfäffikon ZH                             NaN   
4         Winterthur                             NaN   
...              ...                             ...   
657692           NaN                             NaN   
657698           NaN                             NaN   
657704           NaN                             NaN   
657718           NaN                             NaN   
657720           NaN                             NaN   

         Endkunde_Branchengruppe Endkunde_Branchengruppe_ID  \
                            last                       last   
0            WG - Dienstleistung                        720   
1          WG - Finanzwirtschaft                        470   
2            WG - Dienstleistung                        720   
3            WG - Dienstleistung                        720   
4       WG - Persönlicher Bedarf                        555   
...                          ...                        ...   
657692                       NaN                        NaN   
657698                       NaN                        NaN   
657704                       NaN                        NaN   
657718                       NaN                        NaN   
657720                       NaN                        NaN   

       Auftrag_Branchengruppe_ID         Kamp_Erfass_Jahr        
                         collect nunique              min   max  
0                            NaN     NaN             2009  2019  
1                            NaN     NaN             2009  2019  
2                            NaN     NaN             2011  2011  
3                            NaN     NaN             2011  2015  
4                            NaN     NaN             2009  2009  
...                          ...     ...              ...   ...  
657692                       980     1.0              NaN   NaN  
657698                       436     1.0              NaN   NaN  
657704                       720     1.0              NaN   NaN  
657718                       720     1.0              NaN   NaN  
657720                       509     1.0              NaN   NaN  

[83888 rows x 12 columns]

### Distribution of Auftragsart vs. Vertrag

In [8]:
display(bd.pivot_table(index=['Auftragsart', 'Vertrag'], columns='Kamp_Erfass_Jahr',aggfunc='size', fill_value=0))
display('Netto-Umsatz')
display(bd.pivot_table(index=['Auftragsart', 'Vertrag'], columns='Kamp_Erfass_Jahr', values='Netto', aggfunc='sum', fill_value=0))

,Kamp_Erfass_Jahr,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Auftragsart,Vertrag,,,,,,,,,,,,,
Aushangauftrag Partner,Nein,0,90,2074,1698,1815,2337,2786,1520,33,16,19,23,15
Eigenwerbung APG,Nein,0,0,294,69,124,177,41,721,1256,1380,1168,423,168
Freespace,Nein,0,0,0,0,0,0,0,0,0,0,2786,4069,1885
Goodwill,Nein,0,10,103,104,123,180,163,134,192,134,150,158,71
Karitativ,Nein,0,200,438,695,403,704,524,726,897,1301,1354,1541,823
Kommerziell,Nein,0,14150,71746,78234,84716,78514,77696,91767,108510,126063,125936,126379,82562
Logistik für Dritte,Nein,0,0,96,525,1434,1416,1292,1530,1900,1866,2045,1772,962
Politisch,Nein,0,1041,3880,4385,8983,5950,5655,6170,10853,9185,6759,7973,7746
Promotion,Nein,0,191,3895,2870,3123,3301,5370,2590,3584,4673,5462,7906,1175


'Netto-Umsatz'

,Kamp_Erfass_Jahr,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Auftragsart,Vertrag,,,,,,,,,,,,,
Aushangauftrag Partner,Nein,0,30444,849600,791206,739514,1026394,1000669,439344,110412,92214,117954,150966,81126
Eigenwerbung APG,Nein,0,0,3897655,0,4670,1048532,0,360,133390,126540,1222454,72427,0
Freespace,Nein,0,0,0,0,0,0,0,0,0,0,0,0,0
Goodwill,Nein,0,0,6543,7939,4953,18668,2994,4762,4191,1732,17772,16741,9174
Karitativ,Nein,0,724644,1960077,1628917,1866012,2668615,2253943,2318474,2578634,3459117,2775404,3962032,2657962
Kommerziell,Nein,0,33806952,148051046,162393013,168628594,175031787,158519973,182560236,193519765,204222602,197001081,191750167,165496589
Logistik für Dritte,Nein,0,0,0,0,4600,184578,31589,132394,479850,420927,418245,522661,266499
Politisch,Nein,0,2040430,6938949,6463858,18562173,11082670,9945225,10458567,16977360,11414298,6379246,8489070,11226187
Promotion,Nein,0,1083041,5375644,4376195,3393104,3505266,5451654,3185928,3787542,4991362,4881049,9304271,2263649
